In [1]:
import os

In [2]:
%pwd

'd:\\Machine_Learning\\Text-Summarizer\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\Machine_Learning\\Text-Summarizer'

In [4]:
# Creation of the entity : the return type of a function is the entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_url: str
    local_data_file: Path
    unzip_dir:Path

In [5]:
# Importing the config and params file path and the necessary functions

from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [6]:
# Creating the configuration Manager

class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

        
    
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion 

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_url= config.source_url,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

In [8]:
# Getting the packages to create the components
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [9]:
# Creating the components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        # If the data does not exits it will download from the link provided
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with the following \n {headers}")
        else:
            logger.info(f"File already exits of size {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        zip_file_path:str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
# Creating the pipeline
try:
    config =ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-23 16:11:46,447: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-23 16:11:46,448: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-23 16:11:46,452: INFO: common: created directory at: artifacts]
[2023-12-23 16:11:46,461: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-23 16:11:54,206: INFO: 3994593015: artifacts/data_ingestion/data.zip downloaded! with the following 
 Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: ABC8:5C7D:162DA45:1AE0DC9:6586B969
Accept-Ranges: bytes
Date: Sat, 23 Dec 2023 10:41:47 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10226-MAA
X-Cache: 

In [ ]:
# Now convert them to Modular Program